### Semantic Chunker의 원리 알아보기

SemanticChunker는 LangChain에서 제공하는 텍스트 분할 기능 중 하나로, 임베딩모델만 있으면 쉽게 적용할 수 있습니다.

그러나 하나씩 로직을 구현해보면서 SemanticChunker의 원리를 이해해보겠습니다.

#### 📄 Step 1. PDF 불러오기

In [ ]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 경로 설정 (사용자 파일로 변경하세요)
pdf_path = "data\국가별 공공부문 AI 도입 및 활용 전략.pdf"

# PDF 문서 로드
loader = PyPDFLoader(pdf_path)
pages = loader.load()
text = "\n".join([p.page_content for p in pages])

#### ✂️ Step 2. 문장 단위로 자르기

In [ ]:
import re

# 기본적인 정규표현식 기반 문장 분리
sentences_raw = re.split(r'(?<=[.?!])\s+', text)
sentences = [{'sentence': s, 'index': i} for i, s in enumerate(sentences_raw)]

#### 🔁 Step 3. 인접 문장 결합 (context noise 완화)

✅ 왜? → 너무 짧은 문장은 의미가 희미해서 임베딩이 불안정함


→ 앞뒤 문장을 함께 포함시켜 "의미 밀도"를 높임

In [ ]:
def combine_sentences(sentences, buffer=1):
    for i in range(len(sentences)):
        combined = ''
        for j in range(i - buffer, i + buffer + 1):
            if 0 <= j < len(sentences):
                combined += sentences[j]['sentence'] + ' '
        sentences[i]['combined'] = combined.strip()
    return sentences

# 개선
# 문장 하나만 사용하면 임베딩이 불안정해질 수 있음
# → 앞뒤 문장을 함께 결합하여 의미 밀도를 높이고,
#    코사인 거리 기반 분할 시 더 안정적인 의미 단절 판단 가능하게 만듦
sentences = combine_sentences(sentences)

In [ ]:
sentences[1]

#### 🔍 Step 4. 임베딩 생성

In [ ]:
from langchain_ollama import OllamaEmbeddings
embedding_model = OllamaEmbeddings(model="bge-m3")

embeddings = embedding_model.embed_documents([s['combined'] for s in sentences])

# 각 문장 dict에 임베딩 추가
for i, emb in enumerate(embeddings):
    sentences[i]['embedding'] = emb

print(sentences[0])

#### 🧮 Step 5. 문장 간 코사인 거리 계산

✅ 목적: 앞뒤 문장의 의미 차이를 수치화하여 단절점 탐색

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

distances = []
for i in range(len(sentences) - 1):
    sim = cosine_similarity([sentences[i]['embedding']], [sentences[i + 1]['embedding']])[0][0]
    distance = 1 - sim
    distances.append(distance)
    sentences[i]['distance_to_next'] = distance

#### 📊 Step 6. 거리 시각화(한글 포함)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

# ✅ 시스템별 한글 폰트 설정 (Windows: 맑은 고딕, macOS: AppleGothic, Linux: 나눔고딕)
import platform
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
else:
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

threshold = np.percentile(distances, 90)

# 코사인 거리 시각화
plt.figure(figsize=(12, 4))
plt.plot(distances)
plt.axhline(threshold, color='red', linestyle='--', label='90% 임계값')
plt.title("문장 간 임베딩 거리")
plt.xlabel("문장 위치")
plt.ylabel("코사인 거리")
plt.legend()
plt.grid(True)
plt.show()

#### ✂️ Step 7. 의미 기반 청크 생성

✅ 거리 값이 급격히 커지는 지점을 단락 구분점으로 간주


In [ ]:
break_points = [i for i, d in enumerate(distances) if d > threshold]

chunks_manual = []
start = 0
for idx in break_points:
    text = ' '.join([s['sentence'] for s in sentences[start:idx + 1]])
    chunks_manual.append(text)
    start = idx + 1
if start < len(sentences):
    chunks_manual.append(' '.join([s['sentence'] for s in sentences[start:]]))

# 🔍 예시 청크 출력
print("수동 청크 개수:", len(chunks_manual))
print("\n예시 청크 1:\n", chunks_manual[0][:300], "...")

In [ ]:
# 각 청크의 길이 계산 및 출력
chunk_lengths = [len(chunk) for chunk in chunks_manual]
print("각 청크별 길이:", chunk_lengths)

# 특정 청크 내용 출력 
print("\n<3번째 청크 내용>")
print("-" * 80)  # 구분선 추가
print(chunks_manual[2])
print("-" * 80)  # 구분선 추가

print("\n<4번째 청크 내용>")
print("-" * 80)  # 구분선 추가
print(chunks_manual[3])
print("-" * 80)  # 구분선 추가

In [ ]:
print(chunks_manual[3])

### 🧠 LangChain의 SemanticChunker로 자동 분할


In [ ]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 경로 설정 (사용자 파일로 변경하세요)
pdf_path = "data\국가별 공공부문 AI 도입 및 활용 전략.pdf"

# PDF 문서 로드
loader = PyPDFLoader(pdf_path)
pages = loader.load()
text = "\n".join([p.page_content for p in pages])

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

# SemanticChunker 적용
semantic_chunker = SemanticChunker(embedding_model, 
    breakpoint_threshold_type="percentile", 
    breakpoint_threshold_amount=85)
    
sem_chunks = semantic_chunker.create_documents([text])

print("\nSemanticChunker 청크 수:", len(sem_chunks))
print("\n예시 청크 1:\n", sem_chunks[0].page_content[:300])

In [ ]:
len(sem_chunks)

In [ ]:
[len(chunk.page_content) for chunk in sem_chunks]

In [ ]:
# 각 청크의 길이 계산 및 출력
chunk_lengths = [len(chunk.page_content) for chunk in sem_chunks]
print("각 청크별 길이:", chunk_lengths)

# 특정 청크 내용 출력 
print("\n<3번째 청크 내용>")
print("-" * 80)  # 구분선 추가
print(sem_chunks[2].page_content[-500:])
print("-" * 80)  # 구분선 추가

print("\n<4번째 청크 내용>")
print("-" * 80)  # 구분선 추가
print(sem_chunks[3].page_content[:500])
print("-" * 80)  # 구분선 추가